In [1]:
import pandas as pd
import numpy as np
import datetime as dt

from util_funcs import *

### Loading data

In [2]:
PATH_TO_INPUT_FILES = './data/'

In [3]:
df_pop = pd.read_csv(PATH_TO_INPUT_FILES+'italy_reg_pop_DEF.csv')
df_covid = pd.read_csv(PATH_TO_INPUT_FILES+'italy_cases_recovered_deaths_DEF.csv', parse_dates=['date'])
df_flows = pd.read_csv(PATH_TO_INPUT_FILES+'italy_movement_24_feb_22_apr_DEF.csv', parse_dates=['date'])
df_lockdown = pd.read_csv(PATH_TO_INPUT_FILES+'italy_lockdown_data.csv', parse_dates=['date'])

In [4]:
# all data until 2020-04-22
df_covid = df_covid[df_covid['date'] <= max(df_flows['date'])]
df_lockdown = df_lockdown[df_lockdown['date'] <= max(df_flows['date'])]
df_covid.tail()

,date,reg_name,positives,delta_positives,removed,delta_removed
1175,2020-04-22,TOSCANA,6167,-455,2533,552
1176,2020-04-22,TRENTINO ALTO ADIGE,3386,-59,2676,97
1177,2020-04-22,UMBRIA,371,-36,986,40
1178,2020-04-22,VALLE D'AOSTA,501,-21,594,23
1179,2020-04-22,VENETO,9991,-86,6747,420


### The model


$\Delta S_r(t) = - \beta_r \dfrac{I_r(t) S_r(t)}{N_r(t)}$

$\Delta I_r(t) = \beta_r \bigg{(}I_r(t) + \sum_{r' \ne r} p_{r'} L_{r',r}(t) \dfrac{I_{r'}(t)}{N_{r'}(t)} + o_r(t) \bigg{)} \dfrac{S_r(t)}{N_r(t)} - \gamma_r \dfrac{I_r(t)}{N_r(t)}$

$\Delta R_r(t) = \gamma_r \dfrac{I_r(t)}{N_r(t)}$

where $L_{r',r}(t)$ is the total movement inflow to region $r$ from all the other regions at time $t$, $p_r$ is the probability that someone who is infected will travel, and $o_r(t)$ captures the probability that someone is infected from externas sources (e.g., international travel).

#### Compute $S_r(t)$, $\Delta S_r(t)$

Loaded datasets give us:

$I_r(t)$, $\Delta I_r(t)$, $R_r(t)$, $\Delta R_r(t)$ $\rightarrow$ respectively `positives`, `delta_positives`, `removed`, `delta_removed` in DataFrame `df_covid`

$N_r(t) = N_r$ $\rightarrow$ `pop` in DataFrame `df_pop`

Then, we need to derive:

$S_r(t)$, $\Delta S_r(t)$

In [5]:
def add_pop_to_df(df_covid, df_pop):
    
    df_with_N = df_covid.groupby(['reg_name']).apply(_add_N, df_pop)
    df_with_N = df_with_N[['date', 'reg_name', 'population'] + list(df_covid.columns[2:])]
    
    return df_with_N

def _add_N(df, df_pop):
    
    population = int(df_pop[df_pop['reg_name'] == df['reg_name'].iloc[0]]['pop'])
    array_N = np.repeat(population, df.shape[0]) # N_r is fixed, it does not change with t
    df['population'] = array_N
    
    return df

def add_susceptibles_to_df(df_covid, df_pop):
    
    if 'population' not in df_covid.columns:
        df_covid = add_pop_to_df(df_covid, df_pop).copy()
    
    df_with_S = df_covid.groupby(['reg_name']).apply(_add_S)
    df_with_S = df_with_S[['date', 'reg_name', 'population', 'susceptibles', 'delta_susceptibles'] + list(df_covid.columns[3:])]
    
    return df_with_S

def _add_S(df):
    
    df['susceptibles'] = df['population'] - (df['positives'] + df['removed'])
    
    array_S = np.array(df['susceptibles'])
    array_delta_S = np.insert(np.diff(array_S), 0, array_S[0])
    df['delta_susceptibles'] = array_delta_S
    
    return df

In [6]:
df_all = add_susceptibles_to_df(df_covid, df_pop)

# showing the result for one region...
df_all[df_all['reg_name'] == 'PIEMONTE'].head(10)

,date,reg_name,population,susceptibles,delta_susceptibles,positives,delta_positives,removed,delta_removed
11,2020-02-24,PIEMONTE,4356407,4356404,4356404,3,3,0,0
31,2020-02-25,PIEMONTE,4356407,4356404,0,3,0,0,0
51,2020-02-26,PIEMONTE,4356407,4356404,0,3,0,0,0
71,2020-02-27,PIEMONTE,4356407,4356405,1,2,-1,0,0
91,2020-02-28,PIEMONTE,4356407,4356396,-9,11,9,0,0
111,2020-02-29,PIEMONTE,4356407,4356396,0,11,0,0,0
131,2020-03-01,PIEMONTE,4356407,4356358,-38,49,38,0,0
151,2020-03-02,PIEMONTE,4356407,4356356,-2,51,2,0,0
171,2020-03-03,PIEMONTE,4356407,4356351,-5,56,5,0,0
191,2020-03-04,PIEMONTE,4356407,4356325,-26,82,26,0,0


####  Data aggregation

Covid-19 data are stored at a daily basis.

In our model, we will use aggregated data, then we need an aggregation...

In [7]:
def aggregate_SIR_data(df_covid, num_of_days_per_group = 7):
    
    df_aggr = df_covid.groupby(['reg_name']).apply(_aggregate, num_of_days_per_group).reset_index()
    df_aggr = df_aggr[list(df_covid.columns)]
    
    return df_aggr

def _aggregate(df, num_of_days_per_group):

    # creating the labels for the aggregation
    # these are stored in a temporary new column ('group_labels') of type [0,0,0,1,1,1,2,2,2,...,k,k,k] if the num_of_days_per_group = 3
    num_of_entries = df.shape[0]
    days = np.arange(0,num_of_entries)
    group_labels = np.repeat(days, num_of_days_per_group)[0:num_of_entries]
    df['group_labels'] = group_labels

    # aggregating the deltas (delta_positives and delta_removed)
    df_aggr = df.groupby(['group_labels']).agg({'delta_susceptibles':'sum', 'delta_positives':'sum', 'delta_removed':'sum'}).reset_index()

    # for population, positives and removed, the value of the last day in each aggregated group is the value we want for that group:
    last_indexes = [group_indexes[len(group_indexes)-1] for group_indexes in df.groupby(['group_labels']).groups.values()]
    df_aggr['susceptibles'] = np.array(df.loc[last_indexes, 'susceptibles'])
    df_aggr['positives'] = np.array(df.loc[last_indexes, 'positives'])
    df_aggr['removed'] = np.array(df.loc[last_indexes, 'removed'])
    df_aggr['population'] = np.array(df.loc[last_indexes, 'population'])

    # the date of each aggregated group will be the date of the first day in each group:
    first_indexes = [group_indexes[0] for group_indexes in df.groupby(['group_labels']).groups.values()]
    df_aggr['date'] = np.array(df.loc[first_indexes, 'date'])

    return df_aggr

If we have $N$ days, as we have seen above, the first $k$ of them will be organized like this:

| date  | $N_r$      | $S_r$      | $\Delta S_r$      | $I_r$      | $\Delta I_r$      | $R_r$      | $\Delta R_r$      |
|-------|------------|------------|-------------------|------------|-------------------|------------|-------------------|
| $d_0$ | $N_r(d_0)$ | $S_r(d_0)$ | $\Delta S_r(d_0)$ | $I_r(d_0)$ | $\Delta I_r(d_0)$ | $R_r(d_0)$ | $\Delta R_r(d_0)$ |
| $d_1$ | $N_r(d_1)$ | $S_r(d_1)$ | $\Delta S_r(d_1)$ | $I_r(d_1)$ | $\Delta I_r(d_1)$ | $R_r(d_1)$ | $\Delta R_r(d_1)$ |
|  ...  |            |            |                   |            |                   |            |                   |
| $d_k$ | $N_r(d_k)$ | $S_r(d_k)$ | $\Delta S_r(d_k)$ | $I_r(d_k)$ | $\Delta I_r(d_k)$ | $R_r(d_k)$ | $\Delta R_r(d_k)$ |

If we aggregate the data into $\big{\lceil}\frac{N}{k}\big{\rceil}$ groups of $k$ days, the $k$ rows of the table above will be grouped in the first group, say $D = (d_0, d_1, ..., d_k)$, thus resulting in one row in te aggregated DataFrame:

| date  | $N_r$      | $S_r$                      | $\Delta S_r$    | $I_r$                      | $\Delta I_r$    | $R_r$                      | $\Delta R_r$    |
|-------|------------|----------------------------|-----------------|----------------------------|-----------------|----------------------------|-----------------|
| $d_0$ | $N_r(d_k)$ | $S_r(d_0) + \Delta S_r(D)$ | $\Delta S_r(D)$ | $I_r(d_0) + \Delta I_r(D)$ | $\Delta I_r(D)$ | $R_r(d_0) + \Delta R_r(D)$ | $\Delta R_r(D)$ |

where 

$\Delta S_r(D) = \sum_{i=0}^k \Delta S_r(d_i)$

$\Delta I_r(D) = \sum_{i=0}^k \Delta I_r(d_i)$

$\Delta R_r(D) = \sum_{i=0}^k \Delta R_r(d_i)$

The `date` indicating the group will just be the date of the first day in the group (in the example, $d_0$).

As in our model $N_r(t) = N_r$, the value of population of region $r$ is simply its value in the last day in the group (in the example, $N_r(d_k)$).

In [8]:
df_aggregated = aggregate_SIR_data(df_all, 7)

df_aggregated[df_aggregated['reg_name'] == 'PIEMONTE'].head(10)

,date,reg_name,population,susceptibles,delta_susceptibles,positives,delta_positives,removed,delta_removed
99,2020-02-24,PIEMONTE,4356407,4356358,4356358,49,49,0,0
100,2020-03-02,PIEMONTE,4356407,4356047,-311,355,306,5,5
101,2020-03-09,PIEMONTE,4356407,4355296,-751,1030,675,81,76
102,2020-03-16,PIEMONTE,4356407,4351987,-3309,4127,3097,293,212
103,2020-03-23,PIEMONTE,4356407,4348201,-3786,7268,3141,938,645
104,2020-03-30,PIEMONTE,4356407,4344045,-4156,10177,2909,2185,1247
105,2020-04-06,PIEMONTE,4356407,4339747,-4298,12505,2328,4155,1970
106,2020-04-13,PIEMONTE,4356407,4335350,-4397,14470,1965,6587,2432
107,2020-04-20,PIEMONTE,4356407,4333668,-1682,15122,652,7617,1030


#### Movement flows

In the equations, we need $L_{r',r}(t)$, i.e. the movement flows from $r'$ to $r$ during $t$.

These are stored in the DataFrame `df_flows`, with two levels of spatial aggregation: `provincia` and `regione`.

In [9]:
df_flows.head()

,date,start_prov_name,end_prov_name,flow,start_reg_name,end_reg_name
0,2020-02-24,LIVORNO,PISTOIA,12,TOSCANA,TOSCANA
1,2020-02-24,PARMA,REGGIO NELL'EMILIA,691,EMILIA ROMAGNA,EMILIA ROMAGNA
2,2020-02-24,SALERNO,POTENZA,49,CAMPANIA,BASILICATA
3,2020-02-24,PARMA,MANTOVA,67,EMILIA ROMAGNA,LOMBARDIA
4,2020-02-24,FOGGIA,AVELLINO,13,PUGLIA,CAMPANIA


A few functions are useful to extract movement flows between different regions in arbitrary periods.

For example, lets extract the flow from Toscana to Lazio between the 24th and the 28th of February, i.e. at the very beginning of the crisis:

In [10]:
extract_flow_in_time_interval(df_flows, 'TOSCANA', 'LAZIO',
                              t_0 = dt.datetime(2020, 2, 24, 0, 0, 0), t_1 = dt.datetime(2020, 2, 28, 0, 0, 0),
                              type_of_region = 'regione')

933

The following week (between the 2nd and the 6th of March), the flow decreased by almost 50% :

In [11]:
extract_flow_in_time_interval(df_flows, 'TOSCANA', 'LAZIO',
                              t_0 = dt.datetime(2020, 3, 2, 0, 0, 0), t_1 = dt.datetime(2020, 3, 6, 0, 0, 0),
                              type_of_region = 'regione')

491

And during the second week of March (9th-13th, when the lockdown started), it is decreased by 68% :

In [12]:
extract_flow_in_time_interval(df_flows, 'TOSCANA', 'LAZIO',
                              t_0 = dt.datetime(2020, 3, 9, 0, 0, 0), t_1 = dt.datetime(2020, 3, 13, 0, 0, 0),
                              type_of_region = 'regione')

298

We can also extract in/out-flows from/to a certain region to/from all the other regions (i.e. our $L_{r',r}(t)$ in the model).

E.g. for Lombardia in the second week of March, we have:

In [13]:
inflow = extract_inflow_in_time_interval(df_flows, 'LOMBARDIA', 
                                         t_0 = dt.datetime(2020, 3, 9, 0, 0, 0), t_1 = dt.datetime(2020, 3, 13, 0, 0, 0), 
                                         type_of_region = 'regione')
outflow = extract_outflow_in_time_interval(df_flows, 'LOMBARDIA', 
                                           t_0 = dt.datetime(2020, 3, 9, 0, 0, 0), t_1 = dt.datetime(2020, 3, 13, 0, 0, 0), 
                                           type_of_region = 'regione')
print('In-flow: ', inflow)
print('Out-flow: ', outflow)

In-flow:  10642
Out-flow:  8991


### Modeling the parameters $\beta_r$, $p_r$ and $o_r$

#### The indexes of restrictions
The [Oxford COVID-19 Government Response Tracker](https://covidtracker.bsg.ox.ac.uk/) collects information on common policy responses, scores the stringency of such measures, and aggregates these into a Stringency Index. 

We will use 8 out of the 9 indicators used to compute the Stringency Index:

| $C_j$ | indicator description              |
|-------|------------------------------------|
| $C_1$ | school_closing                     |
| $C_2$ | workplace_closing                  |
| $C_3$ | cancel_public_events               |
| $C_4$ | restrictions_on_gatherings         |
| $C_5$ | close_public_transport             |
| $C_6$ | stay_at_home_requirements          |
| $C_7$ | restrictions_on_internal_ movement |
| $C_8$ | international_travel_controls      |


In [14]:
df_lockdown.tail()

,date,C1,C1_Flag,C2,C2_Flag,C3,C3_Flag,C4,C4_Flag,C5,C5_Flag,C6,C6_Flag,C7,C7_Flag,C8,stringency_index
108,2020-04-18,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,91.8
109,2020-04-19,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,91.8
110,2020-04-20,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,91.8
111,2020-04-21,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,91.8
112,2020-04-22,3.0,1.0,3.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,91.8


The first 6 of them will describe our $\beta_r$, while $C_7$ will be used to model $p_r$, and finally $o_r$ will depend only on $C_8$.

As the indicators are computed at a country-level, we will have $\beta_r = \beta$, $p_r = p$ and $o_r = o$.

Each indicator $I_j$ takes values in an ordinal scale from 0 to $N_j$, and all of them except for $I_8$ also have a flag $G_j \in \{0,1\}$ that indicates whether the policy is generally applied (i.e. to the whole country) or not (i.e. it is applied only on target regions).

They are computed as:

$I_j = C_j \dfrac{1-w}{N_j} + w G_j, \quad j \in \{1,2,...,7\}$

where $w = \frac{1}{7} \sum_{j = 1}^7 \frac{1}{N_j+1}$.

Then, each indicator takes a standardized ‘bonus point’ for a generally-applied policy, and we have $0 \leq I_j \leq 1$. 

The last one is simply:

$I_8 = \frac{C_8}{N_8}$

In [14]:
map_Indicator_to_N = {
    '1': 3,
    '2': 3,
    '3': 2,
    '4': 4,
    '5': 2,
    '6': 3,
    '7': 2,
    '8': 4
}

w = (1/8) * sum([1/(N+1) for N in map_Indicator_to_N.values()])

def compute_I(C, C_num, G=0):
    if C_num != '8':
        I = C * (1-w) / map_Indicator_to_N[C_num] + w * G
    else:
        # this is for indicator 8, that has no flag 1/0
        I = C / map_Indicator_to_N[C_num]
    return I

def add_I_to_df(df_lockdown):
    
    df_with_I = df_lockdown.copy()
    
    for C_num in map_Indicator_to_N.keys():
        if C_num != '8':
            df_with_I['I_'+C_num] = compute_I(df_with_I['C'+C_num], C_num, df_with_I['C'+C_num+'_Flag'])
        else:
            df_with_I['I_'+C_num] = compute_I(df_with_I['C'+C_num], C_num)
        
    return df_with_I.fillna(0) # " [...] an absence of data corresponds to a sub-index of zero."

df_Ind = add_I_to_df(df_lockdown)[['date', 'I_1', 'I_2', 'I_3', 'I_4', 'I_5', 'I_6', 'I_7', 'I_8']]
df_Ind.tail()

,date,I_1,I_2,I_3,I_4,I_5,I_6,I_7,I_8
108,2020-04-18,1.0,1.0,1.0,1.0,1.0,0.75625,1.0,0.5
109,2020-04-19,1.0,1.0,1.0,1.0,1.0,0.75625,1.0,0.5
110,2020-04-20,1.0,1.0,1.0,1.0,1.0,0.75625,1.0,0.5
111,2020-04-21,1.0,1.0,1.0,1.0,1.0,0.75625,1.0,0.5
112,2020-04-22,1.0,1.0,1.0,1.0,1.0,0.75625,1.0,0.5


#### The parameters

- $o_r(t)$ depends only on $C_8$, so we could model it with a simple linear regression:

> $o_r(t) = \alpha_r^o I_8(t) + c_r^o$

- $p_r(t)$ is a probability and depends only on $C_7$, so we can use a logistic model:

> $ln \frac{p_r(t)}{1-p_r(t)} = \alpha_r^p I_7(t) + c_r^p$

- The other restrictions affect the intra-regional infection rate, so we use them to model $\beta_r(t)$:

> $\beta_r(t) = \sum_{i = 1}^6 \alpha_i^{\beta} I_i(t) + c_r^{\beta}$.

### Writing the model ($p_r$ =1)

In [33]:
###### MODEL WITH p_r = 1 #######

from scipy.optimize import minimize

class Learner(object):
    def __init__(self, region, loss, start_date):
        self.region = region
        self.loss = loss
        self.start_date = start_date

    ### functions to load data: ###
    def load_positives(self, region):
        df = df_covid
        region_df = df[df['reg_name'] == region]
        return region_df[region_df['date'] >= start_date]['positives']
    
    def load_delta_positives(self, region):
        df = df_covid
        region_df = df[df['reg_name'] == region]
        return region_df[region_df['date'] >= start_date]['delta_positives']

    def load_removed(self, region):
        df = df_covid
        region_df = df[df['reg_name'] == region]
        return region_df[region_df['date'] >= start_date]['removed']
    
    def load_delta_removed(self, region):
        df = df_covid
        region_df = df[df['reg_name'] == region]
        return region_df[region_df['date'] >= start_date]['delta_removed']
    
    def load_pop(self, region):
        df = df_pop
        region_df = df[df['reg_name'] == region]
        return int(region_df['pop']) ## pop is fixed
    
    def load_index(self, list_index):
        df = df_Ind
        return df[df['date'] >= start_date][list_index]
    
    def load_positives_from_out(self, region, p):
        I_out = 0
        for c_region in set_all_regions:
            if c_region != region:
                ## L
                c_df = df_flows[(df_flows['end_reg_name'] == region) & 
                                   (df_flows['start_reg_name'] == c_region) & 
                                   (df_flows['date'] >= start_date)][['date', 'flow']]
                df_with_inflow = pd.DataFrame(columns=c_df.columns)
                for c_date in set(df_flows[df_flows['date'] >= start_date]['date']):
                    if c_date not in set(c_df['date']):
                        df_with_inflow = df_with_inflow.append({'date': c_date, 'flow': 0}, ignore_index=True)
                    else:
                        daily_aggr_inflow = np.sum(c_df[c_df['date'] == c_date]['flow'])
                        df_with_inflow = df_with_inflow.append({'date': c_date, 'flow': daily_aggr_inflow}, ignore_index=True)

                df_with_inflow = df_with_inflow.sort_values(['date']).reset_index(drop=True)
                L = df_with_inflow['flow']
                
                ## I
                I = df_covid[(df_covid['reg_name'] == c_region) & 
                             (df_covid['date'] >= start_date)]['positives'].reset_index(drop=True)
                
                ## N
                N = int(df_pop[df_pop['reg_name'] == c_region]['pop'])
                
                I_out += p * L * I / N
        
        return I_out


    def train(self):
        
        ### loading data: ###
        removed = self.load_removed(self.region)
        #print('removed, ', removed.shape)
        delta_removed = self.load_delta_removed(self.region)
        #print('delta_removed, ', delta_removed.shape)
        positives = self.load_positives(self.region)
        #print('positives, ', positives.shape)
        delta_positives = self.load_delta_positives(self.region)
        #print('delta_positives, ', delta_positives.shape)
        #susceptibles = positives - removed
        #print('susceptibles, ', susceptibles.shape)
        
        population = self.load_pop(self.region)
        #print('population, ', population)
        
        positives_from_out = self.load_positives_from_out(self.region, p=1)   ### p = 1
        #print('positives_from_out, ', positives_from_out.shape)
        
        restrictions = self.load_index(['I_1', 'I_2', 'I_3', 'I_4', 'I_5', 'I_6', 'I_7', 'I_8'])
        #print('restrictions, ', restrictions.shape)
        
        
        ### minimizing the loss function: ###
        optimal = minimize(loss, [1/6,1/6,1/6,1/6,1/6,1/6,0,1,0], 
                           args=(positives, removed, delta_positives, delta_removed, 
                                 population, positives_from_out, restrictions), 
                           method='BFGS')
        print(optimal)
        alpha_b_1, alpha_b_2, alpha_b_3, alpha_b_4, alpha_b_5, alpha_b_6, c_b, alpha_o, c_o = optimal.x
        

### defining the loss function:
def loss(point, positives, removed, delta_positives, delta_removed, 
         population, positives_from_out, restrictions):
    
    ### parameters: ###
    alpha_b_1, alpha_b_2, alpha_b_3, alpha_b_4, alpha_b_5, alpha_b_6, c_b, alpha_o, c_o = point
    
    ### data: ###
    I = positives
    R = removed
    N = population
    S = N - (I+R)
    I_out = positives_from_out
    
    ### gamma, beta, o ###
    gamma = 1/3
    beta = alpha_b_1*restrictions['I_1'] + alpha_b_2*restrictions['I_2'] + alpha_b_3*restrictions['I_3'] + alpha_b_4*restrictions['I_4'] + alpha_b_5*restrictions['I_5'] + alpha_b_6*restrictions['I_6'] + c_b
    o = alpha_o*restrictions['I_8'] + c_o
    
    ### SIR equations: ###
    dSdt = -beta * I * S / N
    dIdt = beta * ((I + I_out + o) * S) / N - gamma * I / N
    dRdt = gamma * I / N

    ### scores to minimize: ###
    l1 = np.sqrt(np.mean((dIdt - delta_positives)**2))
    l2 = np.sqrt(np.mean((dRdt - delta_removed)**2))
    alpha = 0.1
    
    return alpha * l1 + (1 - alpha) * l2


######


### initialization:
set_all_regions = set(df_pop['reg_name'])
regions, start_date = set_all_regions, dt.datetime(2020, 2, 24, 0, 0, 0)

### all the regions:
'''
for region in regions:
    
    learner = Learner(region, loss, start_date, predict_range)
    learner.train()
'''

### only 1 region:
learner = Learner('TOSCANA', loss, start_date)
learner.train()


      fun: 82.36831456767423
 hess_inv: array([[ 0.95029338, -0.04970662, -0.04970662, -0.04970662,  0.        ,
        -0.08588419, -0.00981979, -0.01559947, -0.02079953],
       [-0.04970662,  0.95029338, -0.04970662, -0.04970662,  0.        ,
        -0.08588419, -0.00981979, -0.01559947, -0.02079953],
       [-0.04970662, -0.04970662,  0.95029338, -0.04970662,  0.        ,
        -0.08588419, -0.00981979, -0.01559947, -0.02079953],
       [-0.04970662, -0.04970662, -0.04970662,  0.95029338,  0.        ,
        -0.08588419, -0.00981979, -0.01559947, -0.02079953],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.08588419, -0.08588419, -0.08588419, -0.08588419,  0.        ,
         0.90757942, -0.07867838, -0.02139979, -0.02853337],
       [-0.00981979, -0.00981979, -0.00981979, -0.00981979,  0.        ,
        -0.07867838,  1.06609966, -0.00920453, -0.01227284],
       [-0.01559947, -